In [1]:
libraries = c("dplyr", "lubridate", "rstan")
for(x in libraries) { library(x,character.only=TRUE,warn.conflicts=FALSE,quietly=TRUE) }

theme_set(theme_minimal(base_size=12)) 

'%&%' = function(x,y)paste0(x,y)

options(mc.cores = parallel::detectCores())
rstan_options(auto_write = TRUE)

packageVersion("rstan")
packageVersion("StanHeaders")
rstan::stan_version()

rstan (Version 2.19.3, GitRev: 2e1f913d3ca3)

For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)



[1] ‘2.19.3’

[1] ‘2.21.0.1’

[1] "2.21.0"

In [6]:
datestar = today()
stanmaindir = '../../../Hokkaido_Backup/TokyoEOC_cats'
unlink(stanmaindir, recursive=T)
dir.create(stanmaindir)

standirname = stanmaindir%&%"/"%&%format(datestar,"%Y%m%d")%&%'-R0'
unlink(standirname, recursive=T)
dir.create(standirname)

"functions {
    vector algebraic_equation(vector y, vector theta,  real[] x_r, int[] x_i) {
      vector[1] z;
      z[1] = expm1(-y[1]*theta[1]) + theta[1];
      return z;
    }
}

data {
    int<lower = 1> N; //total number of samples
    int<lower = 0, upper = N> n; //number of positive samples
}

transformed data {
    real X_r[0];
    int X_i[0];

    vector[1] R0_guess;
    R0_guess[1] = 1.2;
}

parameters {
    real<lower = 0, upper = 1> z; //fraction of positively infected
}

transformed parameters {
    vector[1] theta;
    theta[1] = z;
    real<lower = 0> R0;

    R0 = algebra_solver(algebraic_equation, R0_guess, theta, X_r, X_i)[1];
}

model {
    z ~ normal(0.5, 1.0);
    n ~ binomial(N, z);
}

generated quantities {
    real<lower = 0, upper = 1> p = 1.0 - 1.0/R0; //probability of major outbreak
}" %>% cat(file=standirname %&% "/fit.stan", sep="", fill=TRUE)

# Dumping data
N = 102
n = 15
stan_rdump(c('N', 'n'), file=standirname%&%"/Data.R") 

# Dumping initial conditions
z = 0.1
stan_rdump(c('z'), file=standirname%&%"/Init.R") 

standistribdir = "../../../../CmdStan"
stanscriptdir = "../Dropbox/" %&% substring(standirname,10)

## bash file
"#!/bin/bash
cwd=$(pwd)
cd "%&%standistribdir%&%"
make -j6 "%&%stanscriptdir%&%"/fit
cd "%&%stanscriptdir%&%"
mkdir -p diagnostics
for i in {1..5}
do
    echo Running ${i}
    SEEDNUMBER=$((1+$i))
    ./fit \\
        method=sample num_samples=2000 num_warmup=4000 save_warmup=0 \\
            adapt delta=0.92 \\
            algorithm=hmc \\
                engine=nuts \\
        random seed=${SEEDNUMBER} \\
        id=$i \\
        data file=Data.R \\
        init=Init.R \\
        output file=trace-$i.csv \\
            diagnostic_file=diagnostics/diagnostics-$i.csv > diagnostics/output-$i.txt &
done
echo Finished sampling haha!
" %>% cat(file=standirname%&%"/fit.sh", sep="", fill=TRUE)

# running the bash script
system("bash "%&%standirname%&%"/fit.sh", intern = TRUE)

[1] ""                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
 [2] "--- Translating Stan model to C++ code ---"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
 [3] "bin/stanc  --o=../Dropbox/Hokkaido_Backup/TokyoEOC_cats/20200408-R0/fit.hpp ../Dropbox/Hokkaido_Backup/TokyoEOC_cats/20200408-R0/fit.stan"                                                                                                                                                                                                                             